# Requirements

Loop through each file in the `parsed_files` folder and extract the following data of the records above the ones where `type = so` and `model = SUV LHD` (e.g. if row 10's `type = so` and `model = SUV LHD`, get the data from row 9):

* Participant number
* Trial number
* Time
* Type
* Model
* ID
* speedInKmPerHour
* inIntersection
* offsetFromLaneCenter

## Output location

Save the output to a CSV file named `driving performance_data.csv`.

# Script

Import the os, pandas,  and load_workbook from openpyxl modules.

In [ ]:
import os
import pandas as pd
from openpyxl import load_workbook

Define the paths of the source and destination directories.

In [ ]:
src_path = "parsed_files/"
dst_path = ""

Define the file name CSV file where the output will be saved.

In [ ]:
filename = "driving performance_data"

In [ ]:
# loop through the files in parsed_files
for file in os.listdir(src_path):

    # log which file is being read
    print("------------------------------")
    print("Reading " + file)

    # define the list of required columns (excluding Participant number and Trial number which will be added later).
    column_names = ["Time", "Type", "Model", "ID", "speedInKmPerHour", "inIntersection", "offsetFromLaneCenter"]
    
    # extract the file data and save it to a dataframe
    df = pd.read_csv(src_path + file, engine="python")
    
    # NOTE: The default read_csv formatting is utf-8, which is probably not the
    # format of our csv files since I'm encountering UnicodeDecodeError.
    # SOLUTION: add encoding = "ISO-8859-1" or engine = "python" to the parameters.

    # get the indices of the records where type = so and model = SUV LHD
    indices = df.index[(df["Type"] == "so") & (df["Model"] == "SUV LHD")]

    # check if there are records found in the file
    if len(indices) != 0:

        # log how many records are found
        print("Records found: " + str(len(indices)))
        print("------------------------------\n")

        # get the indices of the rows above the records where type = so and model = SUV LHD
        indices = [i - 1 for i in indices]

        # filter the dataframe to display only the required column names of the selected indices.
        df = df.loc[indices, column_names]

        # add the participant and trials numbers to the list of column names
        column_names.insert(0, "Participant number")
        column_names.insert(1, "Trial number")

        # add the participant and trials numbers to the dataframe
        df.insert(loc = 0, column = column_names[0], value = file[0:2])
        df.insert(loc = 1, column = column_names[1], value = file[3:5])

        df["Participant number"] = df["Participant number"].astype("str")

        # check if the output file doesn't exist yet (meaning, there are no headers yet as well)
        if not os.path.exists(dst_path + filename + ".xlsx"):

            # set the column_names as headers and save df to xlsx
            df.to_excel(dst_path + filename + ".xlsx", header = column_names, index = False)

        # if there's already an existing output file
        else:

            # open the file and load its books and sheets
            writer = pd.ExcelWriter(dst_path + filename + ".xlsx", engine='openpyxl')
            writer.book = load_workbook(dst_path + filename + ".xlsx")
            writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)

            # append the dataframe after the last row excluding the headers
            reader = pd.read_excel(dst_path + filename + ".xlsx")
            df.to_excel(writer, header = False, index = False, startrow=len(reader)+1)

            writer.close()